In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
import time
from multiprocessing import Pool
import numpy as np
import pandas as pd
from math import sqrt
import tensorflow as tf

In [24]:
np.random.seed(42)
train_set = open("ml-1m/ratings.dat").readlines()
train_set = np.random.permutation(train_set)[:100]
train_threshold = int(0.75 * len(train_set))
# train_indices = []
# test_indices = []
train_user_indices = []
train_item_indices = []
test_user_indices = []
test_item_indices = []
train_ratings = []
test_ratings = []

data = defaultdict(dict)
user2id = {}
item2id = {}
index_user = 0
index_item = 0

for i, line in enumerate(train_set):
    user = line.split("::")[0]
    item = line.split("::")[1]
    rating = line.split("::")[2]
    
    try:
        user_id = user2id[user]
    except KeyError:
        user_id = index_user
        user2id[user] = index_user
        index_user += 1
    try:
        item_id = item2id[item]
    except KeyError:
        item_id = index_item
        item2id[item] = index_item
        index_item += 1
        
    if i < train_threshold:
    #    train_indices.append((user_id, item_id))
        train_user_indices.append(user_id)
        train_item_indices.append(item_id)
        train_ratings.append(int(rating))
        data[user_id].update(dict(zip([item_id], [int(rating)])))
    else:
    #    test_indices.append((user_id, item_id))
        test_user_indices.append(user_id)
        test_item_indices.append(item_id)
        test_ratings.append(int(rating))

In [25]:
len(train_ratings), len(test_ratings)

(75, 25)

In [26]:
def ratings(dataset):
    for user, r in data.items():
        for item, rating in r.items():
            yield user, item, rating

In [27]:
n_users = len(user2id)
n_items = len(item2id)
n_factors = 100
user_list = np.array(list(data.keys()), dtype=np.int32)

In [32]:
aa = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "rating", "timestamp"])
aa = aa.filter(regex="user|item|rating")

In [36]:
aa.head()

,user,item,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [39]:
aa[aa.user == 1].to_dict()

{'item': {0: 1193,
  1: 661,
  2: 914,
  3: 3408,
  4: 2355,
  5: 1197,
  6: 1287,
  7: 2804,
  8: 594,
  9: 919,
  10: 595,
  11: 938,
  12: 2398,
  13: 2918,
  14: 1035,
  15: 2791,
  16: 2687,
  17: 2018,
  18: 3105,
  19: 2797,
  20: 2321,
  21: 720,
  22: 1270,
  23: 527,
  24: 2340,
  25: 48,
  26: 1097,
  27: 1721,
  28: 1545,
  29: 745,
  30: 2294,
  31: 3186,
  32: 1566,
  33: 588,
  34: 1907,
  35: 783,
  36: 1836,
  37: 1022,
  38: 2762,
  39: 150,
  40: 1,
  41: 1961,
  42: 1962,
  43: 2692,
  44: 260,
  45: 1028,
  46: 1029,
  47: 1207,
  48: 2028,
  49: 531,
  50: 3114,
  51: 608,
  52: 1246},
 'rating': {0: 5,
  1: 3,
  2: 3,
  3: 4,
  4: 5,
  5: 3,
  6: 5,
  7: 5,
  8: 4,
  9: 4,
  10: 5,
  11: 4,
  12: 4,
  13: 4,
  14: 5,
  15: 4,
  16: 3,
  17: 4,
  18: 5,
  19: 4,
  20: 3,
  21: 3,
  22: 5,
  23: 5,
  24: 3,
  25: 5,
  26: 4,
  27: 4,
  28: 4,
  29: 3,
  30: 4,
  31: 4,
  32: 4,
  33: 4,
  34: 4,
  35: 4,
  36: 5,
  37: 5,
  38: 4,
  39: 5,
  40: 5,
  41: 5,
  42: 4

In [41]:
data[1].keys(), data[1].values()

(dict_keys([1]), dict_values([5]))

In [20]:
aa.iloc[0].to_dict().values()

dict_values([1, 1193, 5])

In [21]:
def ratings2(dataset):
    for user, item, rating in zip(dataset.user, dataset.item, dataset.rating):
        yield user, item, rating

In [68]:
aa = np.arange(16).reshape(4,4)
aa

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [96]:
np.take_along_axis(aa[[0,2]], np.array([[0,2]]), axis=1)

array([[ 0,  2],
       [ 8, 10]])

In [100]:
np.take(aa[[0,2]], [0,2], axis=1)

array([[ 0,  2],
       [ 8, 10]])

In [104]:
aa[[0,2]][:, [0,2]]

array([[ 0,  2],
       [ 8, 10]])

In [113]:
aa[[0,2,2], [1,2,3]]

array([ 1, 10, 11])

In [120]:
np.array([i for i in range(10)])

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [118]:
np.take(aa, list(zip(index1, index2)))

array([[0, 1],
       [2, 2],
       [2, 3]])

In [115]:
np.where(aa == 10)

(array([2]), array([2]))

## Collaborative Filtering

In [ ]:
# UNDO :  common calculate   raw_id2inner_id  pearson_similarity  predict_ratings

In [5]:
def common(x1,x2):
    # find common ratings
    common = (x1 != 0) & (x2 != 0)
    new_x1 = x1[common]
    new_x2 = x2[common]
    return new_x1, new_x2

In [6]:
x = np.array([1,2,3,0])
x1 = np.array([1,0, 4, 6])
c = (x != 0) & (x1 != 0)
x[c], x1[c]

(array([1, 3]), array([1, 4]))

In [2]:
def common(x1, x2):
    common = (x1 != 0) & (x2 != 0)
    new_x1 = x1[common]
    new_x2 = x2[common]
    return new_x1, new_x2

In [16]:
x1 = np.array([4,0,5,5])
x2 = np.array([2,1,3,5])
common(x1, x2)

(array([4, 5, 5]), array([2, 3, 5]))

In [17]:
list(zip(common(x1, x2)[0], common(x1, x2)[1]))

[(4, 2), (5, 3), (5, 5)]

In [18]:
numerator = 0
denominator1 = 0
denominator2 = 0
mean1 = np.mean(common(x1, x2)[0])
mean2 = np.mean(common(x1, x2)[1])
for i, j in zip(common(x1, x2)[0], common(x1, x2)[1]):
    numerator += (i - mean1) * (j - mean2)
    denominator1 += (i - mean1) ** 2
    denominator2 += (j - mean2) ** 2

numerator / (np.sqrt(denominator1) * np.sqrt(denominator2))

0.7559289460184546

## matrix factorization

In [ ]:
example = defaultdict(dict)
for key in range(200):
#    key = str(key)
    example[key] = data[key]
# data = example

In [9]:
a = tf.constant(np.array([[1,2,3,0,2], [1,0,3,0,3]]), dtype=tf.float32)
zero = tf.constant(0, dtype=tf.float32)  ## -1
mask = tf.not_equal(a, zero)
# indices = tf.reshape(tf.where(mask), shape=[-1])
indices = tf.where(mask)
c = tf.gather_nd(a, indices)
d = tf.boolean_mask(a, mask)

sess = tf.InteractiveSession()
sess.run([mask, indices, c, d])

[array([[ True,  True,  True, False,  True],
        [ True, False,  True, False,  True]]), array([[0, 0],
        [0, 1],
        [0, 2],
        [0, 4],
        [1, 0],
        [1, 2],
        [1, 4]]), array([1., 2., 3., 2., 1., 3., 3.], dtype=float32), array([1., 2., 3., 2., 1., 3., 3.], dtype=float32)]

In [16]:
def func(a):
    return a[0] + a[1] + a[2]

aa = np.array(train_user_indices)[:10]
bb = np.array(train_item_indices)[:10]
cc = np.array(train_ratings)[:10]
nu = tf.map_fn(func, [aa, bb, cc], dtype=tf.int64)
sess = tf.InteractiveSession()
sess.run(nu)

array([ 2,  7,  8, 10,  9, 14, 17, 17, 17, 19])

In [17]:
aa = np.array([[1,2,3,0,0], [4,5,6,7,8]], dtype=np.int32)
oo = tf.nn.embedding_lookup(aa, 0)
zero = tf.constant(0, dtype=tf.int32)
mask = tf.not_equal(oo, zero)
u_items_mask = tf.boolean_mask(oo, mask)
# ii = tf.gather(oo, u_items_mask)
sess = tf.InteractiveSession()
sess.run([u_items_mask, tf.size(u_items_mask)])

[array([1, 2, 3], dtype=int32), 3]

In [18]:
train_data = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_items = list(data[u].keys())
    train_data[u] = np.array(u_items + [0] * (n_items - len(u_items)))

pseudo_data = tf.placeholder(tf.int32, shape=[n_users, n_items])
yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))

def nu_func(user):
    u_items = tf.nn.embedding_lookup(pseudo_data, user)
    zero = tf.constant(0, dtype=tf.int32)
    mask = tf.not_equal(u_items, zero)
    u_items_mask = tf.boolean_mask(u_items, mask)
    return tf.reduce_sum(tf.gather(yj, u_items_mask), axis=0)

user_list = np.array(list(data.keys()), dtype=np.int32)
nu = tf.map_fn(nu_func, user_list, dtype=tf.float32)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run([nu, tf.shape(nu)], feed_dict={pseudo_data: train_data})

[array([[-0.0599637 ,  0.10894931,  0.02212552, ...,  0.06748607,
         -0.17401433, -0.09068567],
        [ 0.01508748,  0.01031978,  0.00728436, ...,  0.02157799,
         -0.03474513, -0.06860533],
        [ 0.0499043 ,  0.04905506, -0.09098595, ...,  0.2249795 ,
          0.03475139, -0.01830734],
        ...,
        [-0.02819956, -0.00509941,  0.00060968, ..., -0.02477363,
         -0.00529699,  0.01257202],
        [-0.01476342, -0.00543445, -0.00674025, ..., -0.0633116 ,
         -0.00543289,  0.00431837],
        [-0.0382369 ,  0.04862677, -0.00541936, ...,  0.00024505,
          0.03966409,  0.00907592]], dtype=float32),
 array([6040,  100], dtype=int32)]

## Factorization Meets the Neighborhood

In [19]:
aa = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "rating", "timestamp"])
aa = aa.filter(regex="user|item|rating")

In [20]:
aa.iloc[0].to_dict().values()

dict_values([1, 1193, 5])

In [21]:
def ratings2(dataset):
    for user, item, rating in zip(dataset.user, dataset.item, dataset.rating):
        yield user, item, rating

In [25]:
train_data = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_items = list(data[u].keys())
    train_data[u] = np.array(u_items + [-1] * (n_items - len(u_items)))

In [31]:
aa = tf.placeholder(tf.int32, shape=[None, len(item2id)])
oo = tf.nn.embedding_lookup(aa, 1)
zero = tf.constant(-1, dtype=tf.int32)
mask = tf.not_equal(oo, zero)
u_items_mask = tf.boolean_mask(oo, mask)
sess = tf.InteractiveSession()
sess.run(u_items_mask, feed_dict={aa: train_data})

array([   1, 1371, 1992,  202,  139,  269,   15, 1176,  533, 2200, 1124,
        282,  591,  989,  224,  737, 1763,  804,  549, 1063, 3049, 1519,
       1457, 2696, 2345, 1019], dtype=int32)

In [32]:
aa = tf.constant([[1,2,3], [4,5,6]])
bb = tf.gather(tf.gather(aa, 0), [1,1])
test_user = tf.placeholder(tf.int32, shape=[None])
test_item = tf.placeholder(tf.int32, shape=[None])
# elems = (np.array([1,2,3]), np.array([4,5,6]))
def func(x):
    q = x[0]
    r = x[1]
    return q * r

cc = tf.map_fn(func, (test_user, test_item), dtype=tf.int32)

# lambda x: x[0] + x[1]
# elems = (np.array([1, 2, 3], dtype=np.int32), np.array([-1, 1, -1], dtype=np.int32))
# alternate = tf.map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int32)
sess = tf.InteractiveSession()
sess.run(cc, feed_dict={test_user: [1,2,3], test_item: [9,5,6]})

array([ 9, 10, 18], dtype=int32)

In [7]:
import itertools
list(itertools.product([1,2,3], [4,5,6]))

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]

In [10]:
%%time
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c_u = np.zeros((n_users, n_items))
for u, i in zip(train_user_indices, train_item_indices):
    u_items = list(data[u].keys())
    c_ij = tf.reduce_sum(tf.gather(tf.gather(c, i), u_items)) / \
           tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
    c_u[u, i] = c_ij
    
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(c_u)

KeyboardInterrupt: 

#### embedding_lookup

In [96]:
aa = tf.constant([[1,2,3,4,5], [4,5,6,3,9]])
bb = tf.nn.embedding_lookup(aa, [[0,1], [1,1]])
user = tf.gather(bb, 0)
yj = tf.reduce_sum(user ,axis=0)
sess = tf.InteractiveSession()
sess.run([user, yj])

[array([[1, 2, 3, 4, 5],
        [4, 5, 6, 3, 9]], dtype=int32),
 array([ 5,  7,  9,  7, 14], dtype=int32)]

In [11]:
aa = tf.constant([[1,2,3], [4,5,6]])
bb = tf.nn.embedding_lookup(aa, [[0,1]])
sess = tf.InteractiveSession()
sess.run(bb)

array([[[1, 2, 3],
        [4, 5, 6]]], dtype=int32)

#### embedding_lookup_sparse

In [88]:
aa = np.arange(10).reshape(2,5).astype(float)
aa

array([[0., 1., 2., 3., 4.],
       [5., 6., 7., 8., 9.]])

In [90]:
indices = [[0,0], [0,0], [1,1], [2,1], [2,2], [2,2]]
values = [0,1,1,1,1,1]
dense_shape = [2,15]
st = tf.SparseTensor(indices, values, dense_shape)
# aa = tf.constant([[1,2,3], [4,5,6]], dtype=tf.float32)
bb = tf.nn.embedding_lookup_sparse(aa, st, sp_weights=None)
sess = tf.InteractiveSession()
sess.run(bb)

array([[2.5, 3.5, 4.5, 5.5, 6.5],
       [5. , 6. , 7. , 8. , 9. ],
       [5. , 6. , 7. , 8. , 9. ]])

In [84]:
aa = np.arange(27).reshape(3,3,3).astype(float)
aa

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.]],

       [[ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.]],

       [[18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.]]])

In [86]:
indices = [[0,0], [0,1], [0,1], [2,1], [2,2], [3,2]]
values = [1,2,1,1,1,2]
dense_shape = [2,15]
st = tf.SparseTensor(indices, values, dense_shape)
# aa = tf.constant([[1,2,3], [4,5,6]], dtype=tf.float32)
bb = tf.nn.embedding_lookup_sparse(aa, st, sp_weights=None)
sess = tf.InteractiveSession()
sess.run(bb)

array([[[12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.]],

       [[18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.]]])

In [108]:
N = [[] for u in range(n_users)]
for u, i, in zip(train_user_indices, train_item_indices):
    N[u].append(i)

sparse = {'indices': [], 'values': []}
for i, user in enumerate(N):
    for j, item in enumerate(user):
        sparse['indices'].append((i, j))
        sparse['values'].append(item)
        
sparse['dense_shape'] = (n_users, n_items)
implicit_feedback = tf.SparseTensor(**sparse)

yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
yjs = tf.nn.embedding_lookup_sparse(yj, implicit_feedback, sp_weights=None, combiner='sqrtn')
yju = tf.gather(yjs, [2,2,3])
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run([yju, tf.shape(yju)])

[array([[-0.00455127,  0.01736358, -0.0011284 ,  0.00255089, -0.00957593,
         -0.0020925 , -0.01266749, -0.00326786,  0.00396166, -0.00969134,
         -0.00675968, -0.00487921, -0.00627048,  0.00205449, -0.00103079,
          0.00094458, -0.00496926, -0.0083558 ,  0.00159714, -0.01686201,
         -0.00210911, -0.00651083, -0.00908738,  0.01984688,  0.00605965,
         -0.01005583, -0.01515545, -0.00346797, -0.00997632, -0.01489756,
          0.00344885, -0.00133769,  0.00312323,  0.01420122,  0.01937641,
         -0.00676895,  0.00130137,  0.00923016,  0.00342921,  0.00403274,
         -0.01719966,  0.00515556,  0.00074134, -0.00503118,  0.00809505,
          0.00481872, -0.00567423,  0.00239443,  0.00779041,  0.02053146,
          0.00215945,  0.00928877, -0.00847241, -0.00300206, -0.00708658,
          0.00164711,  0.00276376,  0.00022526, -0.01156578, -0.00452333,
         -0.00070281, -0.00081126, -0.00629355,  0.01388548,  0.02596579,
          0.00765122,  0.02379264,  0.

## Dataset API

In [2]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while 1:
            print(sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("End!")

1.0
2.0
3.0
4.0
5.0
End!


In [13]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": np.array([1.0, 2.0, 3.0, 4.0, 5.0]), 
        "b": np.random.uniform(size=(5,2))
    })

In [14]:
dataset.output_types, dataset.output_shapes

({'a': tf.float64, 'b': tf.float64},
 {'a': TensorShape([]), 'b': TensorShape([Dimension(2)])})

In [19]:
c = tf.constant([4.0, 2.0, 3.0])
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
sess.run([one_element['b'], tf.gather(c, tf.cast(one_element['a'], tf.int32))])

[array([0.83989501, 0.6755826 ]), 2.0]

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 2)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
try:
    while True:
        print(sess.run(one_element))
except tf.errors.OutOfRangeError:
    print("End!")

3.0
4.0
5.0
6.0
7.0
End!


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.permutation(np.arange(32*3)))
dataset = dataset.batch(32)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
try:
    while True:
        print(sess.run(one_element))
except tf.errors.OutOfRangeError:
    print("End!")

[40  4 87 58 84 79 93 90 24  6 88 32 43 36 53 81 31 14 45 52 12 61 21 85
 38 22 20 69 29 65 17 44]
[92  9 73 62  2  5 10 82 68 25 37 13 49 57 89 18 95 46 48 33 72 86 63 26
 91  8  7 56 51 41 11 94]
[55 16 47 35 15 50 39 19 23 74  1 34 60 76 77 66 75  0 30 59 28 78  3 67
 70 80 42 71 54 83 64 27]
End!


In [43]:
limit = tf.placeholder(dtype=tf.int32, shape=[])
dataset = tf.data.Dataset.from_tensor_slices(tf.range(start=0, limit=limit))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={limit: 10})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

In [48]:
dataset = tf.data.Dataset.from_tensor_slices(
    {"a": tf.random_uniform([4]), 
     "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)
print(dataset.output_shapes)

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


In [3]:
iterator = tf.data.Iterator.from_structure(tf.int64, tf.TensorShape([]))
dataset_range = tf.data.Dataset.range(10)
range_initializer = iterator.make_initializer(dataset_range)
dataset_evens = dataset_range.filter(lambda x: x % 2 == 0)
evens_initializer = iterator.make_initializer(dataset_evens)

def model_fn(a):
    return a * 2, a * 10

sess = tf.InteractiveSession()
prediction, loss = model_fn(iterator.get_next())
for _ in range(3):
    sess.run(range_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break
    
    sess.run(evens_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print("twooooo")
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break

0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90
0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90
0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90


In [15]:
#　iterator = tf.data.Iterator.from_structure(tf.int32, tf.TensorShape([None]))
# dataset_range = tf.data.Dataset.range(10)

dataset_range = tf.data.Dataset.from_tensor_slices(np.arange(10, dtype=np.int32))
dataset_range = dataset_range.batch(2)
iterator = tf.data.Iterator.from_structure(dataset_range.output_types, dataset_range.output_shapes)
range_initializer = iterator.make_initializer(dataset_range)
dataset_evens = dataset_range.filter(lambda x: x % 2 == 0)
evens_initializer = iterator.make_initializer(dataset_evens)

def model_fn(a):
    return a * 2, a * 10

sess = tf.InteractiveSession()
prediction, loss = model_fn(iterator.get_next())
for _ in range(1):
    sess.run(range_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break
    
    sess.run(evens_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print("twooooo")
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break

[0 2] [ 0 10]
[4 6] [20 30]
[ 8 10] [40 50]
[12 14] [60 70]
[16 18] [80 90]


In [2]:
max_value =tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value
    print(value)

0
1
2
3
4
5
6
7
8
9


## Dataset Module

In [133]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from Dataset.Dataset import Dataset

In [3]:
ml = Dataset.load_dataset(data_path="ml-1m/ratings.dat")
ml_data = Dataset()
ml_data.build_dataset(ml)

In [5]:
LOOV = ml_data.train_test_split_LOOV(k=2)

In [88]:
users, user_position, user_counts = np.unique(ml_data.user_indices, return_inverse=True, return_counts=True)

In [89]:
user_indices = np.split(np.argsort(user_position, kind="mergesort"), np.cumsum(user_counts)[:-1])

In [90]:
user_indices[0].shape, user_indices[0]

((53,),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52]))

In [22]:
user_counts

array([ 53, 129,  51, ...,  20, 123, 341])

In [131]:
user_set = set(ml_data.user_indices)
train_user_indices = []
test_user_indices = []
train_item_indices = []
test_item_indices = []
train_ratings = []
test_ratings = []
train_data = defaultdict(dict)
test_data = defaultdict(dict)
# for u in range(len(user_set)):
for u in ml_data.data.keys():
    user_length = len(user_indices[u])
    if user_length <= k:
        p = 1
    else:
        p = k
    train_indices = user_indices[u][:-p]
    test_indices = user_indices[u][-p:]
    
    train_user_indices.extend(ml_data.user_indices[train_indices])
    test_user_indices.extend(ml_data.user_indices[test_indices])
    train_item_indices.extend(ml_data.item_indices[train_indices])
    test_item_indices.extend(ml_data.item_indices[test_indices])
    train_ratings.extend(ml_data.ratings[train_indices])
    test_ratings.extend(ml_data.ratings[test_indices])
    
for u, i, r in zip(train_user_indices, train_item_indices, train_ratings):
    train_data[u].update(dict(zip([i], [r])))
for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    test_data[u].update(dict(zip([i], [r])))

In [132]:
len(train_data[0])

51

In [113]:
aa = np.array(train_ratings)
from collections import Counter
Counter(aa)

Counter({1: 55645, 2: 106507, 3: 258472, 4: 344765, 5: 222740})

In [114]:
bb = np.array(test_ratings)
from collections import Counter
Counter(bb)

Counter({1: 529, 2: 1050, 3: 2725, 4: 4206, 5: 3570})